In [141]:
import pandas as pd

In [142]:
# csv 파일 불러오기
df = pd.read_csv("서울시 관광특구 주요공원 리스트.csv")

# 컬럼 이름 변경
df = df.rename(columns={
    "구": "gu",
    "공원명": "park_name",
    "네이버맵 기준 블로그 리뷰 수": "navermap_reviews_count",
    "카카오맵 기준 블로그 리뷰 수": "kakaomap_reviews_count",
    "리뷰 수 총합": "total_reviews"
})

# 용산구 / 광진구 분리
yongsan_df = df[df["gu"] == "용산구"]
gwangjin_df = df[df["gu"] == "광진구"]

# 용산구·광진구를 제외한 나머지
other_df = df[~df["gu"].isin(["용산구", "광진구"])]
# 동일 의미: other_df = df[(df["gu"] != "용산구") & (df["gu"] != "광진구")]

In [143]:
df.head()

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,중구,서울로 7017,3282.0,192.0,3474
1,중구,남산공원,1216.0,124.0,1340
2,중구,서소문역사공원,396.0,45.0,441
3,중구,손기정체육공원,254.0,24.0,278
4,중구,훈련원근린공원(훈련원공원),104.0,10.0,114


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gu                      135 non-null    object 
 1   park_name               135 non-null    object 
 2   navermap_reviews_count  129 non-null    float64
 3   kakaomap_reviews_count  129 non-null    float64
 4   total_reviews           135 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 5.4+ KB


In [145]:
# 구 별로 total_review를 기준으로 정렬
df_sorted = df.sort_values(["gu", "total_reviews"], ascending=[True, False])

In [146]:
df.head()

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,중구,서울로 7017,3282.0,192.0,3474
1,중구,남산공원,1216.0,124.0,1340
2,중구,서소문역사공원,396.0,45.0,441
3,중구,손기정체육공원,254.0,24.0,278
4,중구,훈련원근린공원(훈련원공원),104.0,10.0,114


In [147]:
# 구별 Q3 이상 공원만 선택할 수 있게 하는 함수
def select_q3(group):
    q3 = group["total_reviews"].quantile(0.75)
    return group[group["total_reviews"] >= q3]

In [148]:
# 전체 기준: 상위 15% 이상인 공원 추출
q85 = df["total_reviews"].quantile(0.85)
highly_reviewed_df = df[df["total_reviews"] >= q85]

In [149]:
highly_reviewed_df

,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
6,용산구,용산가족공원,3745.0,401.0,4146
8,종로구,인왕산도시자연공원(인왕산),8249.0,331.0,8580
9,종로구,낙산공원,6192.0,296.0,6488
10,종로구,북한산국립공원(북한산),3993.0,126.0,4119
20,송파구,올림픽공원,22371.0,1546.0,23917
21,송파구,송파나루근린공원(석촌호수),20825.0,1146.0,21971
33,마포구,경의선숲길,4935.0,34.0,4969
41,은평구,북한산국립공원(북한산),3993.0,126.0,4119
47,서대문구,북한산국립공원(북한산),3993.0,126.0,4119
51,성북구,북한산국립공원(북한산),3993.0,126.0,4119


In [150]:
# 구별 Q3 기준 + 전체 상위 15% 병합
q3_selection = other_df.groupby("gu", group_keys=False).apply(select_q3)

# 중복 제거 + 병합
final_df = pd.concat([q3_selection, yongsan_df, gwangjin_df, highly_reviewed_df], 
                     ignore_index=True).drop_duplicates()

/var/folders/2l/hl66_jcn0_9d8l_bdq3bjxwc0000gn/T/ipykernel_97487/4050431853.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  q3_selection = other_df.groupby("gu", group_keys=False).apply(select_q3)


In [151]:
final_df["gu"].nunique()

25

In [152]:
# 1. gu별 행 개수 세기
gu_counts = final_df["gu"].value_counts()

# 2. 행이 1개 이하인 gu 찾기
target_gus = gu_counts[gu_counts <= 1].index
print("행이 1개 이하인 구:", list(target_gus))

# 3. 해당 gu에서 total_reviews 기준 2등 뽑기 (원본 df 기준!)
rows_to_add = []
for g in target_gus:
    candidates = df[df["gu"] == g].sort_values("total_reviews", ascending=False)
    if len(candidates) >= 2:   # 원본에서 2등이 존재해야만 추가
        row_2nd = candidates.iloc[1]
        rows_to_add.append(row_2nd)

# 4. final_df에 추가
if rows_to_add:
    final_df = pd.concat([final_df, pd.DataFrame(rows_to_add)], ignore_index=True)

# 최종 확인: 모든 gu가 2개 이상 포함되었는지 체크
check_counts = final_df["gu"].value_counts()
print(check_counts)

행이 1개 이하인 구: ['성북구', '동대문구', '노원구', '금천구', '구로구']
gu
강서구     3
종로구     3
강남구     2
서초구     2
용산구     2
중랑구     2
중구      2
은평구     2
영등포구    2
양천구     2
송파구     2
성북구     2
성동구     2
서대문구    2
강동구     2
마포구     2
동작구     2
동대문구    2
도봉구     2
노원구     2
금천구     2
구로구     2
관악구     2
강북구     2
광진구     2
Name: count, dtype: int64


In [153]:
# 저장
final_df.to_csv("selected_final_parks.csv", index=False)

print(f"최종 선택된 공원 수: {len(final_df)}개")
final_df

최종 선택된 공원 수: 52개


,gu,park_name,navermap_reviews_count,kakaomap_reviews_count,total_reviews
0,강남구,도산근린공원,2081.0,26.0,2107
1,강남구,율현공원,222.0,30.0,252
2,강동구,길동생태공원,466.0,37.0,503
3,강동구,일자산허브천문공원,997.0,63.0,1060
4,강북구,북서울꿈의숲,6556.0,587.0,7143
5,강서구,방화근린공원,510.0,61.0,571
6,강서구,우장산근린공원(우장산/우장산근린공원),711.0,8.0,719
7,강서구,서울식물원,21360.0,2706.0,24066
8,관악구,서울대공원,11538.0,1187.0,12725
9,구로구,푸른수목원,5595.0,163.0,5758
